# Name(s)
**PUT YOUR FULL NAME(S) HERE**

## Should we grade this notebook? (Answer yes or no)

???YES OR NO???

**Instructions:** Pair programming assignment. Submit only a single notebook unless you deviate significantly after lab on Thursday. If you submit individually, make sure you indicate who you worked with originally. Make sure to include your first and last names. For those students who push to individual repos but still work in groups, please indicate which notebook should be graded.

# Recommendation Systems

## Lab Assignment

This is a pair programming assignment. I strongly
discourage individual work for this (and other team/pair programming) lab(s), even if you think you can do it
all by yourself. Also, this is a pair programming assignment, not a ”work in teams of two” assignment. Pair
programming requires joint work on all aspects of the project without delegating portions of the work to individual
1
team members. For this lab, I want all your work — discussion, software development, analysis of the results,
report writing — to be products of joint work.
Students enrolled in the class can pair with other students enrolled in the class. Students on the waitlist can
pair with other students on the waitlists. In the cases of ”odd person out” situations, a team of three people can
be formed, but that team must (a) ask and answer one additional question, and (b) work as a pair would, without
delegation of any work off-line.

## At the end of this lab, I should be able to
* Understand how item-item and user-user collaborative filtering perform recommendations
* Explain a experiment where we tested item-item versus user-user

In [ ]:
# We need a better version
!pip install -U scikit-learn

## Our data
We will be using a well known movielens dataset (small version).

### Here are all the imports that I've used

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

In [2]:
ratings = pd.read_csv('~/csc-466-student/data/movielens-small/ratings.csv') # you might need to change this path

In [3]:
ratings = ratings.dropna()
ratings

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


In [4]:
len(ratings.userId.unique())

610

In [5]:
movies = pd.read_csv('~/csc-466-student/data/movielens-small/movies.csv')

In [6]:
movies = movies.dropna()
movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


### Joining the data together
We need to join those two source dataframes into a single one called data. I do this by setting the index to movieId and then specifying an ``inner`` join which means that the movie has to exist on both sides of the join. Then I reset the index so that I can later set the multi-index of userId and movieId. The results of this are displayed below. Pandas is awesome, but it takes some getting used to how everything works.

In [7]:
data = movies.set_index('movieId').join(ratings.set_index('movieId'),how='inner').reset_index()
#data["movieId"] = data["title"]+" "+data["movieId"].astype(str)
data = data.set_index(['userId','movieId'])[["rating"]]
data

rating
userId movieId        
1      1           4.0
5      1           4.0
7      1           4.5
15     1           2.5
17     1           4.5
...                ...
184    193581      4.0
       193583      3.5
       193585      3.5
       193587      3.5
331    193609      4.0

[100836 rows x 1 columns]

### Turning data into a matrix instead of a series
The functions ``stack()`` and ``unstack()`` are called multiple times in this lab. They allow me to easily change from a dataframe to a series and back again. Below I'm changing from the Series object to a DataFrame. The important thing to note is that each row is now a user! NaN values are inserted where a user did not rate movie.

In [8]:
data=data.unstack()
data

rating                                                                 \
movieId 1      2      3      4      5      6      7      8      9      10       
userId                                                                          
1          4.0    NaN    4.0    NaN    NaN    4.0    NaN    NaN    NaN    NaN   
2          NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
3          NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
4          NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
5          4.0    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
...        ...    ...    ...    ...    ...    ...    ...    ...    ...    ...   
606        2.5    NaN    NaN    NaN    NaN    NaN    2.5    NaN    NaN    NaN   
607        4.0    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
608        2.5    2.0    2.0    NaN    NaN    NaN    NaN    NaN    NaN    4.0   
609        3.0    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    4.0   
610        5.0    NaN    NaN    NaN    NaN    5.0    NaN    NaN    NaN    NaN   

         ...                                                                 \
movieId  ... 193565 193567 193571 193573 193579 193581 193583 193585 193587   
userId   ...                                                                  
1        ...    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
2        ...    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
3        ...    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
4        ...    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
5        ...    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
...      ...    ...    ...    ...    ...    ...    ...    ...    ...    ...   
606      ...    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
607      ...    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
608      ...    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
609      ...    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
610      ...    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   

                
movieId 193609  
userId          
1          NaN  
2          NaN  
3          NaN  
4          NaN  
5          NaN  
...        ...  
606        NaN  
607        NaN  
608        NaN  
609        NaN  
610        NaN  

[610 rows x 9724 columns]

In [63]:
data.columns

MultiIndex([('rating',      1),
            ('rating',      2),
            ('rating',      3),
            ('rating',      4),
            ('rating',      5),
            ('rating',      6),
            ('rating',      7),
            ('rating',      8),
            ('rating',      9),
            ('rating',     10),
            ...
            ('rating', 193565),
            ('rating', 193567),
            ('rating', 193571),
            ('rating', 193573),
            ('rating', 193579),
            ('rating', 193581),
            ('rating', 193583),
            ('rating', 193585),
            ('rating', 193587),
            ('rating', 193609)],
           names=[None, 'movieId'], length=9724)

In [66]:
# So it seems like having this extra 'rating' string is confusing folks a bit
# I'm going to just change it, but leave it as a tuple because it is important to realize that tuples can be indices and columns
data.columns = pd.MultiIndex.from_tuples([('movie',v[1]) for v in data.columns])
data.columns

MultiIndex([('movie',      1),
            ('movie',      2),
            ('movie',      3),
            ('movie',      4),
            ('movie',      5),
            ('movie',      6),
            ('movie',      7),
            ('movie',      8),
            ('movie',      9),
            ('movie',     10),
            ...
            ('movie', 193565),
            ('movie', 193567),
            ('movie', 193571),
            ('movie', 193573),
            ('movie', 193579),
            ('movie', 193581),
            ('movie', 193583),
            ('movie', 193585),
            ('movie', 193587),
            ('movie', 193609)],
           length=9724)

## Let's take a look at some useful code together before the exercises.

First let's look at code that centers the data (important for cosine distance) and then fills in missing values as 0.

In [67]:
data.shape

(610, 9724)

In [68]:
data.mean().shape

(9724,)

In [69]:
data_centered = (data.T-data.mean(axis=1)).T
data_centered = data_centered.fillna(0)
data_centered

movie                                                               \
          1         2         3      4      5         6         7      8        
userId                                                                          
1      -0.366379  0.000000 -0.366379    0.0    0.0 -0.366379  0.000000    0.0   
2       0.000000  0.000000  0.000000    0.0    0.0  0.000000  0.000000    0.0   
3       0.000000  0.000000  0.000000    0.0    0.0  0.000000  0.000000    0.0   
4       0.000000  0.000000  0.000000    0.0    0.0  0.000000  0.000000    0.0   
5       0.363636  0.000000  0.000000    0.0    0.0  0.000000  0.000000    0.0   
...          ...       ...       ...    ...    ...       ...       ...    ...   
606    -1.157399  0.000000  0.000000    0.0    0.0  0.000000 -1.157399    0.0   
607     0.213904  0.000000  0.000000    0.0    0.0  0.000000  0.000000    0.0   
608    -0.634176 -1.134176 -1.134176    0.0    0.0  0.000000  0.000000    0.0   
609    -0.270270  0.000000  0.000000    0.0    0.0  0.000000  0.000000    0.0   
610     1.311444  0.000000  0.000000    0.0    0.0  1.311444  0.000000    0.0   

                         ...                                                   \
       9         10      ... 193565 193567 193571 193573 193579 193581 193583   
userId                   ...                                                    
1         0.0  0.000000  ...    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
2         0.0  0.000000  ...    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
3         0.0  0.000000  ...    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
4         0.0  0.000000  ...    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
5         0.0  0.000000  ...    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
...       ...       ...  ...    ...    ...    ...    ...    ...    ...    ...   
606       0.0  0.000000  ...    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
607       0.0  0.000000  ...    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
608       0.0  0.865824  ...    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
609       0.0  0.729730  ...    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
610       0.0  0.000000  ...    0.0    0.0    0.0    0.0    0.0    0.0    0.0   

                             
       193585 193587 193609  
userId                       
1         0.0    0.0    0.0  
2         0.0    0.0    0.0  
3         0.0    0.0    0.0  
4         0.0    0.0    0.0  
5         0.0    0.0    0.0  
...       ...    ...    ...  
606       0.0    0.0    0.0  
607       0.0    0.0    0.0  
608       0.0    0.0    0.0  
609       0.0    0.0    0.0  
610       0.0    0.0    0.0  

[610 rows x 9724 columns]

### Now what if we want to grab a specific user? Let's say we want the one with user ID of 1.

In [70]:
x = data_centered.loc[1]
x

movie  1        -0.366379
       2         0.000000
       3        -0.366379
       4         0.000000
       5         0.000000
                   ...   
       193581    0.000000
       193583    0.000000
       193585    0.000000
       193587    0.000000
       193609    0.000000
Name: 1, Length: 9724, dtype: float64

### Finding neighborhood.
If we are hoping to predict movies for this user, then user-user collaborative filtering says find the ``N`` users that are similar. We should definitely drop out user 1 because it makes no sense to recommend to yourself. We then compute the cosine similarity between this user ``x`` and all other users in the db. We then reverse sort them, and then display the results.

In [71]:
db = data_centered.drop(1)
sims = db.apply(lambda y: (y.values*x.values).sum()/(np.sqrt((y**2).sum())*np.sqrt((x**2).sum())),axis=1)
sorted_sims = sims.sort_values()[::-1]
sorted_sims

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in double_scalars
  


userId
53          NaN
301    0.124799
597    0.102631
414    0.101348
477    0.099217
         ...   
395   -0.075800
71    -0.082932
370   -0.086678
501   -0.090018
579   -0.105003
Length: 609, dtype: float64

### Grabing similar users
Let's set the network size to 10, and then grab those users :)

In [72]:
N=10
userIds = sorted_sims.iloc[:N].index
data_centered.loc[userIds]

movie                                                           \
          1         2         3      4         5         6         7        
userId                                                                      
53      0.000000  0.000000  0.000000    0.0  0.000000  0.000000  0.000000   
301     0.000000  0.000000  0.000000    0.0  0.000000  0.000000  0.000000   
597     0.022573  0.000000  0.000000    0.0  0.000000 -0.977427 -2.977427   
414     0.608043 -0.391957  0.608043    0.0 -1.391957 -0.391957 -0.391957   
477     0.263333  0.263333 -0.736667    0.0  0.000000  0.000000  0.000000   
57      1.607143  0.000000  0.000000    0.0  0.000000 -0.392857  0.000000   
369     0.000000  0.000000  0.000000    0.0  0.000000  0.000000  0.000000   
206     0.960000  0.000000  0.000000    0.0  0.000000  0.000000 -0.040000   
535     0.000000  0.000000  0.000000    0.0  0.000000  0.000000  0.000000   
590     0.644231 -0.855769 -0.355769    0.0 -1.355769  0.144231  0.000000   

                                   ...                                     \
          8      9         10      ... 193565 193567 193571 193573 193579   
userId                             ...                                      
53      0.000000    0.0  0.000000  ...    0.0    0.0    0.0    0.0    0.0   
301     0.000000    0.0  0.679825  ...    0.0    0.0    0.0    0.0    0.0   
597     0.000000    0.0 -0.977427  ...    0.0    0.0    0.0    0.0    0.0   
414    -0.391957    0.0 -0.391957  ...    0.0    0.0    0.0    0.0    0.0   
477     0.000000    0.0  0.000000  ...    0.0    0.0    0.0    0.0    0.0   
57      0.000000    0.0 -0.392857  ...    0.0    0.0    0.0    0.0    0.0   
369     0.000000    0.0  0.000000  ...    0.0    0.0    0.0    0.0    0.0   
206     0.000000    0.0  0.000000  ...    0.0    0.0    0.0    0.0    0.0   
535     0.000000    0.0  0.000000  ...    0.0    0.0    0.0    0.0    0.0   
590     0.000000    0.0  0.144231  ...    0.0    0.0    0.0    0.0    0.0   

                                           
       193581 193583 193585 193587 193609  
userId                                     
53        0.0    0.0    0.0    0.0    0.0  
301       0.0    0.0    0.0    0.0    0.0  
597       0.0    0.0    0.0    0.0    0.0  
414       0.0    0.0    0.0    0.0    0.0  
477       0.0    0.0    0.0    0.0    0.0  
57        0.0    0.0    0.0    0.0    0.0  
369       0.0    0.0    0.0    0.0    0.0  
206       0.0    0.0    0.0    0.0    0.0  
535       0.0    0.0    0.0    0.0    0.0  
590       0.0    0.0    0.0    0.0    0.0  

[10 rows x 9724 columns]

### How about a prediction?
We could compute the mean from the neighborhood for each prediction

In [73]:
db.loc[userIds].mean()+data.loc[1].mean()

movie  1         4.776912
       2         4.267940
       3         4.317940
       4         4.366379
       5         4.091607
                   ...   
       193581    4.366379
       193583    4.366379
       193585    4.366379
       193587    4.366379
       193609    4.366379
Length: 9724, dtype: float64

### What if we want to weight by the distance?

In [74]:
db.loc[userIds].multiply(sorted_sims.iloc[:N],axis=0).sum()/sorted_sims.iloc[:N].sum()+data.loc[1].mean()

movie  1         4.807995
       2         4.262365
       3         4.316777
       4         4.366379
       5         4.070800
                   ...   
       193581    4.366379
       193583    4.366379
       193585    4.366379
       193587    4.366379
       193609    4.366379
Length: 9724, dtype: float64

## User-user small dataset example

In [75]:
# grab some movies that were watched a lot
r=(data > 0).sum()
our_movies = r.sort_values(ascending=False).iloc[:10].index
our_movies

MultiIndex([('movie',  356),
            ('movie',  318),
            ('movie',  296),
            ('movie',  593),
            ('movie', 2571),
            ('movie',  260),
            ('movie',  480),
            ('movie',  110),
            ('movie',  589),
            ('movie',  527)],
           )

In [76]:
our_data = data[our_movies] # grab only those movies

In [77]:
# Now grab just the users
our_users = (our_data>0).sum(axis=1).sort_values(ascending=False).iloc[:10].index

In [80]:
test_data = our_data.loc[our_users]
test_data

movie                                             
        356  318  296  593  2571 260  480  110  589  527 
userId                                                   
610      3.0  3.0  5.0  4.5  5.0  5.0  5.0  4.5  5.0  3.5
561      4.0  5.0  4.5  4.0  4.0  4.5  4.5  5.0  4.0  4.0
247      5.0  4.5  4.0  4.0  3.5  4.0  4.5  4.0  3.5  4.0
580      4.0  5.0  5.0  4.5  5.0  4.0  4.0  4.5  4.0  3.0
254      5.0  4.5  5.0  4.0  5.0  4.0  4.0  4.0  4.0  5.0
274      4.5  4.5  5.0  4.0  4.0  3.0  3.5  4.5  4.5  4.0
288      5.0  5.0  5.0  5.0  3.0  5.0  2.0  5.0  4.0  5.0
304      5.0  4.0  5.0  4.0  4.0  5.0  4.0  5.0  5.0  5.0
307      4.0  4.5  4.5  4.5  3.5  3.5  3.5  3.5  2.5  4.5
391      5.0  5.0  5.0  3.0  5.0  5.0  2.0  5.0  4.0  5.0

It doesn't serve our purpose to have no missing values, so let's put some back in.

In [81]:
test_data.iloc[0,8] = np.NaN
test_data.iloc[1,8] = np.NaN
test_data.iloc[0,6] = np.NaN
test_data.iloc[5,8] = np.NaN
test_data.iloc[0,2] = np.NaN
test_data.iloc[3,8] = np.NaN
test_data.loc[274,('movie',593)] = np.NaN
test_data.loc[274,('movie',527)] = np.NaN

In [82]:
test_data

movie                                             
        356  318  296  593  2571 260  480  110  589  527 
userId                                                   
610      3.0  3.0  NaN  4.5  5.0  5.0  NaN  4.5  NaN  3.5
561      4.0  5.0  4.5  4.0  4.0  4.5  4.5  5.0  NaN  4.0
247      5.0  4.5  4.0  4.0  3.5  4.0  4.5  4.0  3.5  4.0
580      4.0  5.0  5.0  4.5  5.0  4.0  4.0  4.5  NaN  3.0
254      5.0  4.5  5.0  4.0  5.0  4.0  4.0  4.0  4.0  5.0
274      4.5  4.5  5.0  NaN  4.0  3.0  3.5  4.5  NaN  NaN
288      5.0  5.0  5.0  5.0  3.0  5.0  2.0  5.0  4.0  5.0
304      5.0  4.0  5.0  4.0  4.0  5.0  4.0  5.0  5.0  5.0
307      4.0  4.5  4.5  4.5  3.5  3.5  3.5  3.5  2.5  4.5
391      5.0  5.0  5.0  3.0  5.0  5.0  2.0  5.0  4.0  5.0

In [83]:
test_data = (test_data.T-test_data.T.mean()).T # mean center everything
test_data.loc[610].mean() # check the mean of user 610

2.5376526277146434e-16

In [84]:
x_raw = test_data.loc[610] # x_raw is a user
x_raw

movie  356    -1.071429
       318    -1.071429
       296          NaN
       593     0.428571
       2571    0.928571
       260     0.928571
       480          NaN
       110     0.428571
       589          NaN
       527    -0.571429
Name: 610, dtype: float64

In [85]:
data_raw = test_data # keep a copy of test_data that doesn't have any missing values
test_data = test_data.fillna(0) # fill in missing values

In [86]:
# we need to split this up into training and test sets
train_movies, test_movies = train_test_split(x_raw.dropna(),test_size=0.2,random_state=1)
display(train_movies)
display(test_movies)

movie  318    -1.071429
       356    -1.071429
       260     0.928571
       2571    0.928571
       110     0.428571
Name: 610, dtype: float64

movie  527   -0.571429
       593    0.428571
Name: 610, dtype: float64

In [87]:
# but we just wanted the movies and not the ratings
train_movies, test_movies = train_test_split(x_raw.dropna().index,test_size=0.2,random_state=1)
print('Training movies')
display(train_movies)
print('Testing movies')
display(test_movies)

Training movies


MultiIndex([('movie',  318),
            ('movie',  356),
            ('movie',  260),
            ('movie', 2571),
            ('movie',  110)],
           )

Testing movies


MultiIndex([('movie', 527),
            ('movie', 593)],
           )

In [88]:
test_data

movie                                                              \
            356       318       296       593       2571      260       480    
userId                                                                         
610    -1.071429 -1.071429  0.000000  0.428571  0.928571  0.928571  0.000000   
561    -0.388889  0.611111  0.111111 -0.388889 -0.388889  0.111111  0.111111   
247     0.900000  0.400000 -0.100000 -0.100000 -0.600000 -0.100000  0.400000   
580    -0.333333  0.666667  0.666667  0.166667  0.666667 -0.333333 -0.333333   
254     0.550000  0.050000  0.550000 -0.450000  0.550000 -0.450000 -0.450000   
274     0.357143  0.357143  0.857143  0.000000 -0.142857 -1.142857 -0.642857   
288     0.600000  0.600000  0.600000  0.600000 -1.400000  0.600000 -2.400000   
304     0.400000 -0.600000  0.400000 -0.600000 -0.600000  0.400000 -0.600000   
307     0.150000  0.650000  0.650000  0.650000 -0.350000 -0.350000 -0.350000   
391     0.600000  0.600000  0.600000 -1.400000  0.600000  0.600000 -2.400000   

                                  
            110   589       527   
userId                            
610     0.428571  0.00 -0.571429  
561     0.611111  0.00 -0.388889  
247    -0.100000 -0.60 -0.100000  
580     0.166667  0.00 -1.333333  
254    -0.450000 -0.45  0.550000  
274     0.357143  0.00  0.000000  
288     0.600000 -0.40  0.600000  
304     0.400000  0.40  0.400000  
307    -0.350000 -1.35  0.650000  
391     0.600000 -0.40  0.600000

In [89]:
db = test_data.drop(x_raw.name) # remove this user
db

movie                                                              \
            356       318       296       593       2571      260       480    
userId                                                                         
561    -0.388889  0.611111  0.111111 -0.388889 -0.388889  0.111111  0.111111   
247     0.900000  0.400000 -0.100000 -0.100000 -0.600000 -0.100000  0.400000   
580    -0.333333  0.666667  0.666667  0.166667  0.666667 -0.333333 -0.333333   
254     0.550000  0.050000  0.550000 -0.450000  0.550000 -0.450000 -0.450000   
274     0.357143  0.357143  0.857143  0.000000 -0.142857 -1.142857 -0.642857   
288     0.600000  0.600000  0.600000  0.600000 -1.400000  0.600000 -2.400000   
304     0.400000 -0.600000  0.400000 -0.600000 -0.600000  0.400000 -0.600000   
307     0.150000  0.650000  0.650000  0.650000 -0.350000 -0.350000 -0.350000   
391     0.600000  0.600000  0.600000 -1.400000  0.600000  0.600000 -2.400000   

                                  
            110   589       527   
userId                            
561     0.611111  0.00 -0.388889  
247    -0.100000 -0.60 -0.100000  
580     0.166667  0.00 -1.333333  
254    -0.450000 -0.45  0.550000  
274     0.357143  0.00  0.000000  
288     0.600000 -0.40  0.600000  
304     0.400000  0.40  0.400000  
307    -0.350000 -1.35  0.650000  
391     0.600000 -0.40  0.600000

In [90]:
movie = ('movie',527) # pick a movie in our test set
display(db)
# We should remove any users that did not rate the movie we are interested in predicting. How would including them help us?
db_subset = db.loc[np.isnan(data_raw.drop(x_raw.name)[movie])==False]
display(db_subset)

movie                                                              \
            356       318       296       593       2571      260       480    
userId                                                                         
561    -0.388889  0.611111  0.111111 -0.388889 -0.388889  0.111111  0.111111   
247     0.900000  0.400000 -0.100000 -0.100000 -0.600000 -0.100000  0.400000   
580    -0.333333  0.666667  0.666667  0.166667  0.666667 -0.333333 -0.333333   
254     0.550000  0.050000  0.550000 -0.450000  0.550000 -0.450000 -0.450000   
274     0.357143  0.357143  0.857143  0.000000 -0.142857 -1.142857 -0.642857   
288     0.600000  0.600000  0.600000  0.600000 -1.400000  0.600000 -2.400000   
304     0.400000 -0.600000  0.400000 -0.600000 -0.600000  0.400000 -0.600000   
307     0.150000  0.650000  0.650000  0.650000 -0.350000 -0.350000 -0.350000   
391     0.600000  0.600000  0.600000 -1.400000  0.600000  0.600000 -2.400000   

                                  
            110   589       527   
userId                            
561     0.611111  0.00 -0.388889  
247    -0.100000 -0.60 -0.100000  
580     0.166667  0.00 -1.333333  
254    -0.450000 -0.45  0.550000  
274     0.357143  0.00  0.000000  
288     0.600000 -0.40  0.600000  
304     0.400000  0.40  0.400000  
307    -0.350000 -1.35  0.650000  
391     0.600000 -0.40  0.600000

movie                                                              \
            356       318       296       593       2571      260       480    
userId                                                                         
561    -0.388889  0.611111  0.111111 -0.388889 -0.388889  0.111111  0.111111   
247     0.900000  0.400000 -0.100000 -0.100000 -0.600000 -0.100000  0.400000   
580    -0.333333  0.666667  0.666667  0.166667  0.666667 -0.333333 -0.333333   
254     0.550000  0.050000  0.550000 -0.450000  0.550000 -0.450000 -0.450000   
288     0.600000  0.600000  0.600000  0.600000 -1.400000  0.600000 -2.400000   
304     0.400000 -0.600000  0.400000 -0.600000 -0.600000  0.400000 -0.600000   
307     0.150000  0.650000  0.650000  0.650000 -0.350000 -0.350000 -0.350000   
391     0.600000  0.600000  0.600000 -1.400000  0.600000  0.600000 -2.400000   

                                  
            110   589       527   
userId                            
561     0.611111  0.00 -0.388889  
247    -0.100000 -0.60 -0.100000  
580     0.166667  0.00 -1.333333  
254    -0.450000 -0.45  0.550000  
288     0.600000 -0.40  0.600000  
304     0.400000  0.40  0.400000  
307    -0.350000 -1.35  0.650000  
391     0.600000 -0.40  0.600000

In [91]:
# In order to make the cosine similarity work, we need to have the same dimensions in db_subset and x
# But we want to make sure that the test movies are removed because well they are for testing purposes
x = x_raw.copy()
x.loc[test_movies] = np.NaN
x = x.fillna(0)
x

movie  356    -1.071429
       318    -1.071429
       296     0.000000
       593     0.000000
       2571    0.928571
       260     0.928571
       480     0.000000
       110     0.428571
       589     0.000000
       527     0.000000
Name: 610, dtype: float64

In [92]:
# Now we can actually compute the cosine similarity. This apply function is basically just a for loop over each user
sims = db_subset.apply(lambda y: (y.values*x.values).sum()/(np.sqrt((y**2).sum())*np.sqrt((x**2).sum())),axis=1)

In [93]:
N = 2 # Set the neighborhood to 2 and select the users after sorting
sims.sort_values(ascending=False).iloc[:N]

userId
304    0.062963
391    0.012963
dtype: float64

In [94]:
# But we don't want the similarity scores, just the user ids
neighbors = sims.sort_values(ascending=False).iloc[:N].index
neighbors

Int64Index([304, 391], dtype='int64', name='userId')

In [95]:
# How did our neighborhood rank that movie?
test_data.loc[neighbors,movie]

userId
304    0.4
391    0.6
Name: (movie, 527), dtype: float64

In [96]:
# Finally! Here is our prediction (unweighted)
pred = test_data.loc[neighbors,movie].mean()
pred

0.5

In [97]:
# What about weighted?
top_sims = sims.sort_values(ascending=False).iloc[:N]
top_sims

userId
304    0.062963
391    0.012963
dtype: float64

In [98]:
# Here is our prediction with weighting
weighted_pred = test_data.loc[neighbors,movie].multiply(top_sims,axis=0).sum()/top_sims.sum()
weighted_pred

0.43414588785782543

In [99]:
# How does this compare?
actual = x_raw.loc[movie]
actual

-0.5714285714285712

In [100]:
print("MAE of unweighted:",np.abs(actual-pred))
print("MAE of weighted:",np.abs(actual-weighted_pred))

MAE of unweighted: 1.0714285714285712
MAE of weighted: 1.0055744592863967


## Item-item on the same small dataset
Let's review what we have from above that becomes our input

In [101]:
data_raw

movie                                                              \
            356       318       296       593       2571      260       480    
userId                                                                         
610    -1.071429 -1.071429       NaN  0.428571  0.928571  0.928571       NaN   
561    -0.388889  0.611111  0.111111 -0.388889 -0.388889  0.111111  0.111111   
247     0.900000  0.400000 -0.100000 -0.100000 -0.600000 -0.100000  0.400000   
580    -0.333333  0.666667  0.666667  0.166667  0.666667 -0.333333 -0.333333   
254     0.550000  0.050000  0.550000 -0.450000  0.550000 -0.450000 -0.450000   
274     0.357143  0.357143  0.857143       NaN -0.142857 -1.142857 -0.642857   
288     0.600000  0.600000  0.600000  0.600000 -1.400000  0.600000 -2.400000   
304     0.400000 -0.600000  0.400000 -0.600000 -0.600000  0.400000 -0.600000   
307     0.150000  0.650000  0.650000  0.650000 -0.350000 -0.350000 -0.350000   
391     0.600000  0.600000  0.600000 -1.400000  0.600000  0.600000 -2.400000   

                                  
            110   589       527   
userId                            
610     0.428571   NaN -0.571429  
561     0.611111   NaN -0.388889  
247    -0.100000 -0.60 -0.100000  
580     0.166667   NaN -1.333333  
254    -0.450000 -0.45  0.550000  
274     0.357143   NaN       NaN  
288     0.600000 -0.40  0.600000  
304     0.400000  0.40  0.400000  
307    -0.350000 -1.35  0.650000  
391     0.600000 -0.40  0.600000

In [102]:
# We are going to need to transform this
data_raw.T

userId           610       561  247       580   254       274  288  304   307  \
movie 356  -1.071429 -0.388889  0.9 -0.333333  0.55  0.357143  0.6  0.4  0.15   
      318  -1.071429  0.611111  0.4  0.666667  0.05  0.357143  0.6 -0.6  0.65   
      296        NaN  0.111111 -0.1  0.666667  0.55  0.857143  0.6  0.4  0.65   
      593   0.428571 -0.388889 -0.1  0.166667 -0.45       NaN  0.6 -0.6  0.65   
      2571  0.928571 -0.388889 -0.6  0.666667  0.55 -0.142857 -1.4 -0.6 -0.35   
      260   0.928571  0.111111 -0.1 -0.333333 -0.45 -1.142857  0.6  0.4 -0.35   
      480        NaN  0.111111  0.4 -0.333333 -0.45 -0.642857 -2.4 -0.6 -0.35   
      110   0.428571  0.611111 -0.1  0.166667 -0.45  0.357143  0.6  0.4 -0.35   
      589        NaN       NaN -0.6       NaN -0.45       NaN -0.4  0.4 -1.35   
      527  -0.571429 -0.388889 -0.1 -1.333333  0.55       NaN  0.6  0.4  0.65   

userId      391  
movie 356   0.6  
      318   0.6  
      296   0.6  
      593  -1.4  
      2571  0.6  
      260   0.6  
      480  -2.4  
      110   0.6  
      589  -0.4  
      527   0.6

In [103]:
x_raw

movie  356    -1.071429
       318    -1.071429
       296          NaN
       593     0.428571
       2571    0.928571
       260     0.928571
       480          NaN
       110     0.428571
       589          NaN
       527    -0.571429
Name: 610, dtype: float64

In [104]:
train_movies

MultiIndex([('movie',  318),
            ('movie',  356),
            ('movie',  260),
            ('movie', 2571),
            ('movie',  110)],
           )

In [105]:
test_movies

MultiIndex([('movie', 527),
            ('movie', 593)],
           )

In [106]:
# This is the movie we are still trying to predict (i.e., from the testing set we pick the first one)
movie

('movie', 527)

The intuition behind item-item is we want to predict the rating of a movie based on user 610's ratings on similar movies. In other words, if we knew that most similar movies to 527 were 356, 318, and 296, then we would calculate our prediction like this:

In [107]:
# the use of 'rating' is just an artifact of pandas transformations
ids = [('movie',356),('movie',318),('movie',296)]
x_raw.loc[ids]

movie  356   -1.071429
       318   -1.071429
       296         NaN
Name: 610, dtype: float64

In [108]:
# so we could predict like this
x_raw.loc[ids].mean()

-1.0714285714285712

In [109]:
# but wait, why would we even includ movie 296? The above mean ignores this in the calculation,
# so it is better to just prevent this from happening, we can do that when we search the neighborhood!

In [111]:
test_data = data_raw.T.fillna(0)
test_data

userId           610       561  247       580   254       274  288  304   307  \
movie 356  -1.071429 -0.388889  0.9 -0.333333  0.55  0.357143  0.6  0.4  0.15   
      318  -1.071429  0.611111  0.4  0.666667  0.05  0.357143  0.6 -0.6  0.65   
      296   0.000000  0.111111 -0.1  0.666667  0.55  0.857143  0.6  0.4  0.65   
      593   0.428571 -0.388889 -0.1  0.166667 -0.45  0.000000  0.6 -0.6  0.65   
      2571  0.928571 -0.388889 -0.6  0.666667  0.55 -0.142857 -1.4 -0.6 -0.35   
      260   0.928571  0.111111 -0.1 -0.333333 -0.45 -1.142857  0.6  0.4 -0.35   
      480   0.000000  0.111111  0.4 -0.333333 -0.45 -0.642857 -2.4 -0.6 -0.35   
      110   0.428571  0.611111 -0.1  0.166667 -0.45  0.357143  0.6  0.4 -0.35   
      589   0.000000  0.000000 -0.6  0.000000 -0.45  0.000000 -0.4  0.4 -1.35   
      527  -0.571429 -0.388889 -0.1 -1.333333  0.55  0.000000  0.6  0.4  0.65   

userId      391  
movie 356   0.6  
      318   0.6  
      296   0.6  
      593  -1.4  
      2571  0.6  
      260   0.6  
      480  -2.4  
      110   0.6  
      589  -0.4  
      527   0.6

In [118]:
# The following three lines are the same as the single line left below
x = test_data.loc[movie].drop(x_raw.name)
x

userId
561   -0.388889
247   -0.100000
580   -1.333333
254    0.550000
274    0.000000
288    0.600000
304    0.400000
307    0.650000
391    0.600000
Name: (movie, 527), dtype: float64

In [115]:
db_subset = test_data.loc[train_movies].drop(x_raw.name,axis=1)
db_subset

userId           561  247       580   254       274  288  304   307  391
movie 318   0.611111  0.4  0.666667  0.05  0.357143  0.6 -0.6  0.65  0.6
      356  -0.388889  0.9 -0.333333  0.55  0.357143  0.6  0.4  0.15  0.6
      260   0.111111 -0.1 -0.333333 -0.45 -1.142857  0.6  0.4 -0.35  0.6
      2571 -0.388889 -0.6  0.666667  0.55 -0.142857 -1.4 -0.6 -0.35  0.6
      110   0.611111 -0.1  0.166667 -0.45  0.357143  0.6  0.4 -0.35  0.6

In [120]:
sims = db_subset.apply(lambda y: (y.values*x.values).sum()/(np.sqrt((y**2).sum())*np.sqrt((x**2).sum())),axis=1)
sims

movie  318    -0.077769
       356     0.611656
       260     0.266618
       2571   -0.347785
       110    -0.018052
dtype: float64

In [121]:
top_sims = sims.sort_values(ascending=False).iloc[:N]
top_sims

movie  356    0.611656
       260    0.266618
dtype: float64

In [122]:
ids = top_sims.index
ids

MultiIndex([('movie', 356),
            ('movie', 260)],
           )

In [125]:
pred = x_raw.loc[ids].mean()
pred

-0.07142857142857117

In [126]:
weighted_pred = x_raw.loc[ids].multiply(top_sims,axis=0).sum()/top_sims.sum()
weighted_pred

-0.46428844444778783

In [127]:
print("MAE of unweighted:",np.abs(actual-pred))
print("MAE of weighted:",np.abs(actual-weighted_pred))

MAE of unweighted: 0.5
MAE of weighted: 0.10714012698078335


## Finally to the exercises!
I want you to implement user-user, item-item, and a combination of item-item and user-user.

## Exercise 1 (Worth 5 points)
Complete the following function that predicts using user-user collaborative filtering. 

In [33]:
def predict_user_user(data_raw,x_raw,N=10,frac=0.02):
    # data_raw is our uncentered data matrix. We want to make sure we drop the name of the user we
    # are predicting:
    db = data_raw.drop(x_raw.name)
    # We of course want to center and fill in missing values
    db = (db.T-db.T.mean()).fillna(0).T
    # Now this is a little tricky to think about, but we want to create a train test split of the movies
    # that user x_raw.name has rated. We need some of them but want some of them removed for testing.
    # This is where the frac parameter is used. I want you to think about how to select movies for training
    #ix_raw, ix_raw_test = train_test_split(???,test_size=frac,random_state=42) # Got to ignore some movies
    
    # Here is where we use what you figured out above
    x_raw_test = x_raw.loc[ix_raw_test] 
    x_raw = x_raw.copy()
    x_raw.loc[ix_raw_test] = np.NaN # ignore the movies in test
    x = (x_raw - x_raw.mean()).fillna(0)

    preds = []
    for movie in ix_raw_test:
        #sims = db.loc[??? Only look at users who have rated this movie ???].apply(lambda y: (y.values*x.values).sum()/(np.sqrt((y**2).sum())*np.sqrt((x**2).sum())),axis=1)
        sims = sims.dropna()
        try:
            sorted_sims = sims.sort_values()[::-1]
        except:
            preds.append(0) # means there is no one that also rated this movie amongst all other users
            continue
        top_sims = sorted_sims.iloc[:N]
        ids = top_sims.index
        #preds.append(??? using ids how do you predict ???)
    pred = pd.Series(preds,index=x_raw_test.index)
    actual = x_raw_test-x_raw.mean()
    mae = (actual-pred).abs().mean()
    return mae

In [34]:
mae = predict_user_user(data,data.loc[1])
mae

0.8241596814667028

In [35]:
maes = data.head(20).apply(lambda x: predict_user_user(data,x),axis=1)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:24: RuntimeWarning: invalid value encountered in double_scalars


In [36]:
np.mean(maes)

0.8285102602687481

## Exercise 2 (Worth 5 points)
Complete the following function that predicts using item-item collaborative filtering. 

In [37]:
def predict_item_item(data_raw,x_raw,N=10,frac=0.02,debug={}):
    ix_raw, ix_raw_test = train_test_split(x_raw.dropna().index,test_size=frac,random_state=42) # Got to ignore some movies
    x_raw_test = x_raw.loc[ix_raw_test]
    
    db = data_raw.drop(x_raw.name)
    db = (db.T-db.T.mean()).fillna(0).T
    # ??? db = FIX DB SO WE CAN KEEP CODE SIMILAR BUT DO ITEM-ITEM ???
    preds = []
    for movie in ix_raw_test:
        x = db.loc[movie]
        # sims = db.drop(movie).loc[??? ONLY SELECT MOVIES IN TRAINING SET WHICH USER HAS RATED ???].apply(lambda y: (y.values*x.values).sum()/(np.sqrt((y**2).sum())*np.sqrt((x**2).sum())),axis=1)
        sims = sims.dropna()
        sorted_sims = sims.sort_values()[::-1]
        top_sims = sorted_sims.iloc[:N]
        ids = top_sims.index
        #preds.append(??? HOW TO PREDICTION ???)
    pred = pd.Series(preds,index=x_raw_test.index)
    actual = x_raw_test
    mae = (actual-pred).abs().mean()
    return mae

In [38]:
mae = predict_item_item(data,data.loc[1])
mae

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in double_scalars
  from ipykernel import kernelapp as app


0.74

In [39]:
maes = data.head(20).apply(lambda x: predict_item_item(data,x),axis=1)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in double_scalars
  from ipykernel import kernelapp as app


In [40]:
np.mean(maes)

0.6948963317384369

**For this very simple experiment, what method seems better?**

YOUR ANSWER HERE

## Exercise 3 (Worth 5 points)
Create new versions of predict_user_user and predict_item_item, but now perform a weighted prediction as was demonstrated above. Did our accuracy get any better?

In [41]:
def predict_item_item(data_raw,x_raw,N=10,frac=0.02,debug={}):
    ix_raw, ix_raw_test = train_test_split(x_raw.dropna().index,test_size=frac,random_state=42) # Got to ignore some movies
    x_raw_test = x_raw.loc[ix_raw_test]
    
    db = data_raw.drop(x_raw.name)
    db = (db.T-db.T.mean()).fillna(0).T
    # ??? db = FIX DB SO WE CAN KEEP CODE SIMILAR BUT DO ITEM-ITEM ???
    preds = []
    for movie in ix_raw_test:
        x = db.loc[movie]
        # sims = db.drop(movie).loc[??? ONLY SELECT MOVIES IN TRAINING SET WHICH USER HAS RATED ???].apply(lambda y: (y.values*x.values).sum()/(np.sqrt((y**2).sum())*np.sqrt((x**2).sum())),axis=1)
        sims = sims.dropna()
        sorted_sims = sims.sort_values()[::-1]
        top_sims = sorted_sims.iloc[:N]
        ids = top_sims.index
        #preds.append(??? HOW TO PREDICTION ???)
    pred = pd.Series(preds,index=x_raw_test.index)
    actual = x_raw_test
    mae = (actual-pred).abs().mean()
    return mae

def predict_user_user(data_raw,x_raw,N=10,frac=0.02):
    # data_raw is our uncentered data matrix. We want to make sure we drop the name of the user we
    # are predicting:
    db = data_raw.drop(x_raw.name)
    # We of course want to center and fill in missing values
    db = (db.T-db.T.mean()).fillna(0).T
    # Now this is a little tricky to think about, but we want to create a train test split of the movies
    # that user x_raw.name has rated. We need some of them but want some of them removed for testing.
    # This is where the frac parameter is used. I want you to think about how to select movies for training
    #ix_raw, ix_raw_test = train_test_split(???,test_size=frac,random_state=42) # Got to ignore some movies
    
    # Here is where we use what you figured out above
    x_raw_test = x_raw.loc[ix_raw_test] 
    x_raw = x_raw.copy()
    x_raw.loc[ix_raw_test] = np.NaN # ignore the movies in test
    x = (x_raw - x_raw.mean()).fillna(0)

    preds = []
    for movie in ix_raw_test:
        #sims = db.loc[??? Only look at users who have rated this movie ???].apply(lambda y: (y.values*x.values).sum()/(np.sqrt((y**2).sum())*np.sqrt((x**2).sum())),axis=1)
        sims = sims.dropna()
        try:
            sorted_sims = sims.sort_values()[::-1]
        except:
            preds.append(0) # means there is no one that also rated this movie amongst all other users
            continue
        top_sims = sorted_sims.iloc[:N]
        ids = top_sims.index
        #preds.append(??? using ids how do you predict ???)
    pred = pd.Series(preds,index=x_raw_test.index)
    actual = x_raw_test-x_raw.mean()
    mae = (actual-pred).abs().mean()
    return mae


In [42]:
mae = predict_item_item(data,data.loc[1])
mae

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in double_scalars
  from ipykernel import kernelapp as app


0.7490869162431907

In [43]:
mae = predict_user_user(data,data.loc[1])
mae

0.8677183304072604

## Exercise 4 (Worth 5-10 extra credit points for one or both implementions)
Combine in sequence item-item and user-user AND/OR user-user and item-item.

In [219]:
%%capture
## YOUR SOLUTION HERE